I've assembled code/thoughts/steps that may help direct our project

First a line of code that might save us some time -> df.isnull().sum() --> returns null values, by column, for the entire data frame.  value_counts(dropna=False) also includes nan's in the value count, which is helpful.

STEP: Bivariate Analysis:
1) For Continuous Variables: Examine Correlations
2) For Categorical Variables: Evaluate distribution of output for each class

STEP: Based on plots generated by Parker we will have to decide which features may benefit from transformation

STEP: Prepare the data for analysis:
1) Select features to test
2) How to handle missing values - eliminate feature vs. dropping missing rows, or both
3) Categorical variables -> binary
4) Ordinal categorical -> dummify
5) Discuss the presence or lack of 5 conditions for linear regression

STEP: The process for fitting a multivariable linear regression:

In [ ]:
#General scheme:
from sklearn import linear_model
ols = linear_model.LinearRegression()
X = adver[['TV', 'Radio', 'Newspaper']]  
Y = adver['Sales']  # response variable
ols.fit(X, Y)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X, Y)))

#Train-Test-Split -- this has been done for us already via test/train data sets, so just informational:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y_bos, test_size=0.3, random_state=42)
print("R^2 for train set: %f" %ols.score(X_train, y_train))
print("R^2 for test  set: %f" %ols.score(X_test, y_test))
#training error = the error we get applying the model to the same data from which we trained.
#test error = the error that we incur on new data.

#Because sklearn does not have a nice summary output:
import statsmodels.api as sm 
X_add_const = sm.add_constant(X_train)  #add a column with Beta Zero =1 because X_train does not include Beta Zero
#you dont have to do this in sklr, by default it calculates intercept
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())

STEP: Test for multi-collinearity by regressing each (continuous) variable against all the others.  Code (lecture) below:

In [ ]:
continuous_features = ['CRIM', 'INDUS', 'NOX', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
scores = {}  #Initialize a list that will hold the scores for each model fit.
ols2 = LinearRegression()
from sklearn.metrics import r2_score
for feature_name in continuous_features:
    df2     = df.copy()
    feature = df2[feature_name].copy()
    df2.drop(feature_name, axis=1, inplace=True)
    ols2.fit(df2, feature)
    scores[feature_name] = ols2.score(df2, feature)
    
#The R-Squared of each model is stored in scores; it can be plotted with this code:
sns.barplot(x='index', y='R2', data=pd.DataFrame(scores, index=['R2']).T.reset_index())
plt.title('$R^2$ of a continuous feature against the other features')

STEP: Model Evaluation

In [ ]:
residuals = y_test - lm.predict(birthFeatures)
plt.hist(residuals)
#------------------
print('R^2 is equal to %.3f' % lm.score(birthFeatures, weights)))
print('RSS is equal to %.3f' %(np.sum(weights - lm.predict(birthFeatures)) ** 2))  #model explains 57% of the variance
print('The intercept is %.3f' % (lm.intercept_)
print('The slopes are %s' % str(np.round(lm.coef_, 3))))

STEP: Would it make more sense to predict the log(price) instead?

In [ ]:
lm.fit(birthFeatures,np.log(weights))
residuals = np.log(weights) - lm.predict(birthFeatures)
plt.hist(residuals)

STEP: Which grouping of features generates the model with the highest R^2?  -- BRUTE FORCE (would be crazy for 100 features)

In [ ]:
#Pick best 3 predictors, create a model over and over again, record each R^2, pick the best
scores = {} # used to record the R^2 of each 3-feature combinations

for idx, name1 in enumerate(birthFeatures.columns):
        myColumns = birthFeatures.columns[(idx+1):]
        for idx2, name2 in enumerate(myColumns):
            myColumns2 = myColumns[(idx2+1):]
            for idx3, name3 in enumerate(myColumns2):
                X2 = birthFeatures[[name1,name2,name3]]
                lm.fit(X2, np.log(weights))
                scores[(name1,name2,name3)] = lmscore(X2,np.log(weights))

STEP: Get rid of features with Low Variance... they don't contribute to the model

In [ ]:
import sklearn.feature_selection as fs
varthres = fs.VarianceThreshold(threshold=1) #initiate object & variance threshold
x_new = varthres.fit_transform(iris.data) #calculate variance/remove those features that are below the variance threshold
x_new.shape

STEP: Univariate Variable Selection (F-test for Regression / Chi-Sq for Classification)

In [ ]:
fs.chi2(iris.data, iris.target) #returns array of Chi^2 and associated P-values
#Select the best features -- k=2 means select the best 2 features
best2 = fs.SelectKBest(fs.chi2, k=2).fit_transform(iris.data, iris.target)
best2.shape

STEP: RIDGE REGRESSION -- allows you to analyze data even in the presence of severe multicollinearity.
Trades away variance caused by multi-collinearity, but does add some bias
LASSO - similar to Ridge but forces coefficients to zero (i.e. eliminates variables/feature selection)

In [ ]:
from sklearn import linear_model
ridge = linear_model.Ridge(alpha = 1) # create a ridge regression instance
ridge.fit(x, y) # fit data
ridge.coef_, ridge.intercept_ # print out the coefficients
print("The determination of ridge regression is: %.4f" %ridge.score(x, y))

Find the optimal penalization parameter (Ridge) by looping thru alpha values (lambda)

In [ ]:
alpha_100 = np.logspace(0, 8, 100)
coef = []
for i in alpha_100:
    ridge.set_params(alpha = i)
    ridge.fit(x, y)
    coef.append(ridge.coef_)
    
df_coef = pd.DataFrame(coef, index=alpha_100, columns=['TV', 'Radio', 'Newspaper'])
import matplotlib.pyplot as plt
title = 'Ridge coefficients as a function of the regularization'
axes = df_coef.plot(logx=True, title=title)
axes.set_xlabel('alpha')
axes.set_ylabel('coefficients')
plt.show()